# Financial Analytics

Objetivo: realizar uma análise prática de desempenho financeiro, cobrindo margens, fluxo de caixa e variações orçamentárias para apoiar decisões de gestão.

Este estudo avalia: (1) evolução de receitas e custos, (2) margens bruta, operacional e líquida, (3) fluxo de caixa operacional, (4) variância orçamentária (real vs. orçamento) e (5) visualizações para suporte gerencial.

In [ ]:
# 1) Leitura dos dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('default')
sns.set_theme(style='whitegrid')

# Espera-se um arquivo notebooks/financial_data.csv com colunas: 
# date, revenue, cogs, operating_expenses, net_income, cash_in, cash_out, budget_revenue, budget_costs, budget_op_ex
df = pd.read_csv('notebooks/financial_data.csv', parse_dates=['date'])
df = df.sort_values('date').reset_index(drop=True)
df.head()

In [ ]:
# 2) Cálculos: margens, receitas, custos e variância orçamentária
df['gross_profit'] = df['revenue'] - df['cogs']
df['operating_profit'] = df['gross_profit'] - df['operating_expenses']
# Caso não haja net_income, podemos aproximar líquido como operating_profit
df['net_profit'] = df.get('net_income', df['operating_profit'])

# Margens (%)
for num, col in [('gross_margin','gross_profit'), ('operating_margin','operating_profit'), ('net_margin','net_profit')]:
    df[num] = np.where(df['revenue'] != 0, df[col] / df['revenue'], np.nan)

# Fluxo de caixa operacional aproximado
if {'cash_in','cash_out'}.issubset(df.columns):
    df['operating_cash_flow'] = df['cash_in'] - df['cash_out']

# Variância orçamentária (real - orçamento)
if {'budget_revenue','budget_costs'}.issubset(df.columns):
    df['rev_variance'] = df['revenue'] - df['budget_revenue']
    # custos reais = cogs + operating_expenses
    df['actual_costs'] = df['cogs'] + df['operating_expenses']
    df['cost_variance'] = df['actual_costs'] - df['budget_costs']

if {'budget_op_ex'}.issubset(df.columns):
    df['op_ex_variance'] = df['operating_expenses'] - df['budget_op_ex']

df.tail()

In [ ]:
# 3) Visualizações: margens e variâncias orçamentárias
fig, axes = plt.subplots(2, 2, figsize=(12, 8), constrained_layout=True)
ax = axes[0,0]
sns.lineplot(data=df, x='date', y='revenue', label='Receita', ax=ax)
sns.lineplot(data=df, x='date', y='cogs', label='COGS', ax=ax)
sns.lineplot(data=df, x='date', y='operating_expenses', label='Opex', ax=ax)
ax.set_title('Receitas e Custos ao longo do tempo')
ax.set_xlabel('Data'); ax.set_ylabel('Valor')

ax = axes[0,1]
for col, label in [('gross_margin','Bruta'),('operating_margin','Operacional'),('net_margin','Líquida')]:
    if col in df.columns:
        sns.lineplot(data=df, x='date', y=col, label=f'Margem {label}', ax=ax)
ax.set_title('Margens (%)')
ax.set_xlabel('Data'); ax.set_ylabel('% da Receita')

ax = axes[1,0]
if 'operating_cash_flow' in df.columns:
    sns.barplot(data=df, x='date', y='operating_cash_flow', ax=ax, color='#4c78a8')
    ax.set_title('Fluxo de Caixa Operacional')
    ax.set_xlabel('Data'); ax.set_ylabel('Valor')
else:
    ax.axis('off')

ax = axes[1,1]
vars_to_plot = []
if 'rev_variance' in df.columns: vars_to_plot.append(('rev_variance','Variação Receita'))
if 'cost_variance' in df.columns: vars_to_plot.append(('cost_variance','Variação Custos'))
if 'op_ex_variance' in df.columns: vars_to_plot.append(('op_ex_variance','Variação Opex'))
if vars_to_plot:
    tmp = df[['date'] + [v for v,_ in vars_to_plot]].melt('date', var_name='tipo', value_name='valor')
    name_map = {v:k for k,v in [('Variação Receita','rev_variance'),('Variação Custos','cost_variance'),('Variação Opex','op_ex_variance')]}
    tmp['tipo'] = tmp['tipo'].map({k:v for v,k in name_map.items()})
    sns.barplot(data=tmp, x='date', y='valor', hue='tipo', ax=ax)
    ax.axhline(0, color='black', linewidth=1)
    ax.set_title('Variações Orçamentárias')
    ax.set_xlabel('Data'); ax.set_ylabel('Variação (Real - Orçado)')
else:
    ax.axis('off')

plt.show()

## Interpretação e recomendações

- Margens: margens ascendentes indicam eficiência; quedas persistentes sugerem rever precificação, mix e custos.
- Fluxo de caixa: quedas podem sinalizar prazos de recebimento longos ou estoques elevados; monitore capital de giro.
- Orçamento: variância positiva em receita e negativa em custos é favorável; investigue desvios relevantes (>10%).
- Ações sugeridas: renegociar fornecedores, otimizar despesas operacionais, revisar metas de vendas e alocação orçamentária, e implementar rotinas mensais de rolling forecast.